<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [4]</a>'.</span>

In [1]:
# Parameters
DB_PATH = "data/interactions.db"
CONTROL_STATUS_URL = "http://localhost:8000/api/control/status"
OUTPUT_DIR = "D:\\Files\\Code 3\\ChatAI-DataLab\\datalab\\notebooks\\_papermill"


# Control Center Playground


This notebook mirrors the Control Center widgets by querying the FastAPI `/api/control` endpoints and summarizing service health alongside interaction telemetry pulled from SQLite. Use it as a quick sanity check after launching the backend, Ops Deck, and Playground UI.

In [2]:
import asyncio
import importlib.util
import os
import sys
from pathlib import Path

SENTINEL_MARKERS = ("PROJECT_OVERVIEW.md", ".git")

if sys.platform.startswith("win"):
    try:
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    except AttributeError:
        # Older Python versions may not expose WindowsSelectorEventLoopPolicy
        pass

def _detect_repo_root() -> Path:
    env_root = os.environ.get("LAB_ROOT")
    candidates: list[Path] = []
    if env_root:
        candidates.append(Path(env_root).expanduser().resolve())
    candidates.append(Path.cwd().resolve())
    for candidate in candidates:
        current = candidate
        for _ in range(8):
            if any((current / marker).exists() for marker in SENTINEL_MARKERS):
                return current
            if current.parent == current:
                break
            current = current.parent
    return candidates[0]

DEFAULT_ROOT = _detect_repo_root()
if str(DEFAULT_ROOT) not in sys.path:
    sys.path.insert(0, str(DEFAULT_ROOT))

package_init = DEFAULT_ROOT / "datalab" / "__init__.py"
if "datalab" not in sys.modules and package_init.exists():
    spec = importlib.util.spec_from_file_location(
        "datalab", package_init, submodule_search_locations=[str(package_init.parent)]
    )
    module = importlib.util.module_from_spec(spec)
    module.__path__ = [str(package_init.parent)]
    sys.modules["datalab"] = module
    assert spec.loader is not None
    spec.loader.exec_module(module)

from datalab.diagnostics import append_diagnostic_record, iter_diagnostic_records, record_run_metadata, write_snapshot
from datalab.lab_paths import data_path, get_lab_root, lab_path, logs_path

LAB_ROOT = get_lab_root()
DB_PATH = Path(globals().get("DB_PATH", data_path("interactions.db")))
CONTROL_STATUS_URL = globals().get("CONTROL_STATUS_URL", "http://localhost:8000/api/control/status")
OUTPUT_DIR = Path(globals().get("OUTPUT_DIR", lab_path("datalab", "_papermill")))
DIAGNOSTIC_LOG = logs_path("lab-diagnostics.jsonl")

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DB_PATH = DB_PATH.expanduser().resolve()

run_metadata = record_run_metadata(parameters={
    "db_path": str(DB_PATH),
    "control_status_url": CONTROL_STATUS_URL,
    "output_dir": str(OUTPUT_DIR),
})
run_metadata

{'lab_root': 'D:\\Files\\Code 3\\ChatAI-DataLab',
 'lab_root_env': None,
 'cwd': 'D:\\Files\\Code 3\\ChatAI-DataLab\\datalab\\notebooks',
 'python_executable': 'C:/Program Files/Python313/python.exe',
 'db_path': 'D:\\Files\\Code 3\\ChatAI-DataLab\\datalab\\notebooks\\data\\interactions.db',
 'control_status_url': 'http://localhost:8000/api/control/status',
 'output_dir': 'D:\\Files\\Code 3\\ChatAI-DataLab\\datalab\\notebooks\\_papermill',
 'generated_at': '2025-11-17T05:00:11.143381+00:00'}

In [3]:
import json
import sqlite3
from statistics import mean
from urllib import request
from urllib.error import URLError

try:
    import pandas as pd
except ImportError:
    import sys, subprocess, importlib
    print("pandas not found; attempting to install…")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas"])
    pd = importlib.import_module("pandas")

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [4]:
conn = sqlite3.connect(str(DB_PATH))
try:
    df = pd.read_sql_query("SELECT user_prompt_text, created_at FROM interactions", conn)
finally:
    conn.close()

prompt_count = len(df)
latest_prompts = df.tail(3) if prompt_count else pd.DataFrame(columns=['user_prompt_text', 'created_at'])
print(f"Loaded {prompt_count} prompts from {DB_PATH}")
prompt_count, latest_prompts

OperationalError: unable to open database file

In [ ]:
fallback_status = {
    "services": [
        {"name": "backend", "state": "unknown", "runtime": "windows"},
        {"name": "frontend", "state": "unknown", "runtime": "windows"},
        {"name": "datalab", "state": "unknown", "runtime": "linux"},
    ],
    "processes": [],
    "network": {
        "hostname": "offline",
        "platform": "unknown",
        "uptime": 0,
        "bytes_sent": 0,
        "bytes_recv": 0,
        "interfaces": {},
    },
    "logs": {"backend": ["No logs available (fallback)."]},
    "timestamp": 0.0,
}

recent_diagnostics = iter_diagnostic_records(limit=20, log_path=DIAGNOSTIC_LOG)
fetched_payload = None

try:
    with request.urlopen(CONTROL_STATUS_URL, timeout=5.0) as response:
        payload_raw = response.read().decode("utf-8")
        fetched_payload = json.loads(payload_raw)
        append_diagnostic_record(
            category="control-center",
            message="Fetched control status",
            data={"url": CONTROL_STATUS_URL},
        )
except (URLError, TimeoutError, json.JSONDecodeError) as exc:
    append_diagnostic_record(
        category="control-center",
        message="Control status fallback",
        data={"url": CONTROL_STATUS_URL, "error": str(exc)},
    )
    fetched_payload = fallback_status | {
        "fallback_reason": f"Control Center API unavailable ({exc})",
    }
    print(fetched_payload["fallback_reason"])

status_payload = fetched_payload or fallback_status
status_payload.setdefault("logs", {}).setdefault("backend", ["No backend logs captured."])
service_states = [f"{svc['name']} ({svc.get('runtime', 'n/a')}): {svc['state']}" for svc in status_payload.get("services", [])]
if not service_states:
    service_states = ["No service data available"]
service_states

In [ ]:
latencies = status_payload.get("logs", {}).get("backend", [])
sample_latency_ms = mean([len(line) for line in latencies]) if latencies else 0
summary = {
    "prompt_count": int(prompt_count),
    "service_states": service_states,
    "sample_latency_ms": round(sample_latency_ms, 2),
    "lab_root": str(LAB_ROOT),
}
if "fallback_reason" in status_payload:
    summary["fallback_reason"] = status_payload["fallback_reason"]
    summary["recent_diagnostics"] = recent_diagnostics[-3:]

snapshot_path = write_snapshot(summary, snapshot_path=OUTPUT_DIR / "control_center_snapshot.json")
append_diagnostic_record(
    category="control-center",
    message="Snapshot written",
    data={"path": str(snapshot_path)},
)
print(f"Snapshot written to {snapshot_path}")
summary

In [ ]:
recent_diagnostics[-5:] if recent_diagnostics else ["No diagnostics recorded yet."]